In [1]:
# The aim of this module is to calculate features based on dividend history

# File name convention
# R_ = raw extract 
# E_ = Entry files into enrichment layer
# C_ = enriched layer with calculation
# I = insights layer, designed for model baselines
# V_ = validation


In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

DIV = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/R_ALL_Div.pkl') 


#create a baselist for adding on the feature engineered variables
TICKER_LIST = DIV['TickName'].drop_duplicates().reset_index()


In [2]:
# find the last total dividend payout for last n years

from datetime import datetime
from dateutil.relativedelta import relativedelta

yrs_ago1 = datetime.now() - relativedelta(years=1)
yrs_ago3 = datetime.now() - relativedelta(years=3)
yrs_ago5 = datetime.now() - relativedelta(years=5)
yrs_ago10 = datetime.now() - relativedelta(years=10)

DIV_CALC = DIV
# DIV_CALC = DIV[DIV['TickName'].isin(['CBA.AX','WES.AX'])]
# export to validate calculation below
# from datetime import datetime as dt
# dte = dt.now().strftime('%Y%m%d')
# fname = "/Users/joezhou/Downloads/TEST_HIST_DIV_PMT{}.xlsx".format(dte)
# DIV_CALC.to_excel(fname)

DIV_CALC['DateTime'] = pd.to_datetime(DIV_CALC['Date'], format='%Y-%m-%d')
DIV_CALC['Year'] = DIV_CALC['DateTime'].dt.year

DIV_CALC['CALC_DIV_1YR'] = DIV_CALC.loc[DIV_CALC['DateTime'] >= yrs_ago1,['Dividends']].sum(axis=1)
DIV_CALC['CALC_DIV_3YR'] = DIV_CALC.loc[DIV_CALC['DateTime'] >= yrs_ago3,['Dividends']].sum(axis=1)
DIV_CALC['CALC_DIV_5YR'] = DIV_CALC.loc[DIV_CALC['DateTime'] >= yrs_ago5,['Dividends']].sum(axis=1)
DIV_CALC['CALC_DIV_10YR'] = DIV_CALC.loc[DIV_CALC['DateTime'] >= yrs_ago10,['Dividends']].sum(axis=1)


# DIV['CALC_DIV_1YR'].fillna(0, inplace=True)


# DF_DIV_HIST_S2 = DIV.loc[(DIV['DateTime'] >= yrs_ago1)]

# calculate the median as reference point
# DF_DIV_MED = DF_DIV_HIST_S2.groupby(['TickName'])['Dividends'].median().reset_index()
DF_DIV_SUM = DIV_CALC.groupby(['TickName'])['CALC_DIV_1YR','CALC_DIV_3YR','CALC_DIV_5YR','CALC_DIV_10YR'].sum().reset_index()
DF_DIV_MED = DIV_CALC.groupby(['TickName'])['CALC_DIV_1YR','CALC_DIV_3YR','CALC_DIV_5YR','CALC_DIV_10YR'].median().reset_index()
DF_DIV_FREQ = DIV_CALC.groupby(['TickName'])['CALC_DIV_1YR','CALC_DIV_3YR','CALC_DIV_5YR','CALC_DIV_10YR'].count().reset_index()
DF_DIV_STD = DIV_CALC.groupby(['TickName'])['CALC_DIV_1YR','CALC_DIV_3YR','CALC_DIV_5YR','CALC_DIV_10YR'].std().reset_index()


DF_DIV_SUM = DF_DIV_SUM.add_suffix('_AMT')
DF_DIV_MED = DF_DIV_MED.add_suffix('_AMT_MED')
DF_DIV_FREQ = DF_DIV_FREQ.add_suffix('_FREQ')
DF_DIV_STD = DF_DIV_STD.add_suffix('_AMT_STDDEV')

DF_DIV_SUM = DF_DIV_SUM.rename(columns={"TickName_AMT": "TickName"})
DF_DIV_MED = DF_DIV_MED.rename(columns={"TickName_AMT_MED": "TickName"})
DF_DIV_FREQ = DF_DIV_FREQ.rename(columns={"TickName_FREQ": "TickName"})
DF_DIV_STD = DF_DIV_STD.rename(columns={"TickName_AMT_STDDEV": "TickName"})


<ipython-input-2-fc80c54eb52e>:35: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  DF_DIV_SUM = DIV_CALC.groupby(['TickName'])['CALC_DIV_1YR','CALC_DIV_3YR','CALC_DIV_5YR','CALC_DIV_10YR'].sum().reset_index()
<ipython-input-2-fc80c54eb52e>:36: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  DF_DIV_MED = DIV_CALC.groupby(['TickName'])['CALC_DIV_1YR','CALC_DIV_3YR','CALC_DIV_5YR','CALC_DIV_10YR'].median().reset_index()
<ipython-input-2-fc80c54eb52e>:37: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  DF_DIV_FREQ = DIV_CALC.groupby(['TickName'])['CALC_DIV_1YR','CALC_DIV_3YR','CALC_DIV_5YR','CALC_DIV_10YR'].count().reset_index()
<ipython-input-2-fc80c54eb52e>:38: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be de

In [3]:
# Combine all parts together

DIV_INFO = TICKER_LIST.merge(DF_DIV_SUM,how='left',on='TickName').merge(DF_DIV_MED,how='left',on='TickName').merge(DF_DIV_FREQ,how='left',on='TickName').merge(DF_DIV_STD,how='left',on='TickName')

del DF_DIV_SUM, DF_DIV_MED, DF_DIV_FREQ, DF_DIV_STD

In [4]:
# find the number of missing row within output table
NAN_DIV_INFO = DIV_INFO.isna().sum()


from datetime import datetime as dt
dte = dt.now().strftime('%Y%m%d')
fname = "/Users/joezhou/Downloads/NAN_COUNT_R_DIV_INFO_{}.xlsx".format(dte)
NAN_DIV_INFO.to_excel(fname)


In [6]:
# calculate rate of return for the dividend

# DF_DIV_HIST_S = DIV_CALC.groupby(['TickName', "Year"])['Dividends'].sum().reset_index()
# DF_DIV_HIST_S.sort_index(axis=1)



In [7]:
#Note: need to loop this later on for 3mth, 6mth, 12mths and 24mths, or some stochastic process

# DIV12_MEDIAN = DIV12.groupby("TickName")["Dividends"].median()
# DIV12_STD = DIV12.groupby("TickName")["Dividends"].std()
# DIV12_MIN = DIV12.groupby("TickName")["Dividends"].min()
# DIV12_MAX = DIV12.groupby("TickName")["Dividends"].max()

# TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV12_MEDIAN,on='TickName').rename(columns={'Dividends': 'Dividends_Median_12mths'})
# TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV12_STD,on='TickName').rename(columns={'Dividends': 'Dividends_Std_12mths'})
# TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV12_MIN,on='TickName').rename(columns={'Dividends': 'Dividends_Min_12mths'})
# TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV12_MAX,on='TickName').rename(columns={'Dividends': 'Dividends_Max_12mths'})


In [8]:

# calulate flat valuation based on dividend returns
# assuming median returns based on PE ratio or flat 10 years

# m = Number of payments per period (e.g., m=2 for semiannually payments)
# t = Number of years to maturity
# ytm = Yield to maturity (in decimals terms)
# fv = The Bond’s Face Value
# c = Coupon rate (in decimals terms)
    
# bondPrice = ((fv*c/m*(1-(1+ytm/m)**(-m*t)))/(ytm/m)) + fv*(1+(ytm/m))**(-m*t)

In [5]:
DIV_INFO.to_pickle("/Users/joezhou/Downloads/Investment/1 data/E_ALL_Div.pkl")


In [6]:
DIV_INFO.to_excel(r'/Users/joezhou/Downloads/Investment/1 data/E_ALL_Div.xlsx', index=False)
